In [1]:
# Лабораторная работа 3
# Вариант 1
# Используемые методы (KNN,RF,LR)
# Подключение библиотек 
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import fetch_20newsgroups
import nltk
from sklearn.preprocessing import StandardScaler

# Выбор классов
categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'rec.autos']
remove = ('headers', 'footers', 'quotes')

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=35, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=35, categories=categories, remove=remove)
print(twenty_train.data[3])
print(twenty_test.data[3])

nltk.download('punkt')

# Применить стемминг
def stemn(data):
    porter_stemmer = PorterStemmer()
    stem = []
    for text in data:
        nltk_tokens = word_tokenize(text)
        line = ''.join([' ' + porter_stemmer.stem(word) for word in nltk_tokens])
        stem.append(line)
    return stem

porter_stemmer = PorterStemmer()
stem_train = []
for text in twenty_train.data:
    nltk_tokens = word_tokenize(text)
    line = ''
    for word in nltk_tokens:
        line += ' ' + porter_stemmer.stem(word)
    stem_train.append(line)
print(stem_train[0])

stem_test = stemn(twenty_test.data)

Concerning the proposed newsgroup split, I personally am not in favor of
   doing this.  I learn an awful lot about all aspects of graphics by reading
   this group, from code to hardware to algorithms.  I just think making 5
   different groups out of this is a wate, and will only result in a few posts
   a week per group.  I kind of like the convenience of having one big forum
   for discussing all aspects of graphics.  Anyone else feel this way?
   Just curious.

I disagree.  You could learn the same amount by reading all the
split groups, and it would make things easier for those of us
who are less omnivorous.  There is no "waste" in creating news
groups -- its just a bit of shuffling about.  I have no problem
with only a few posts per week per group - I spend too much time
on this as it is.

Yep, you can use any type of UNIX, or maybe VMS, or buy a MAC or something...
  If you want longer filenames for your documents, I heard of a wordprocessor for
windows which let you assign lon

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rewaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 i final got a 24 bit viewer for my povray gener .tga file . it wa written in c by sean malloy and he kindli sent me a copi . he wrote it for the same purpos , to view .tga file use hi speedstar 24 . it onli work with the speedstar 24 and i can not send copi sinc it is not my program . i believ the author may releas a version at a futur time when the program is more develop . he may or may not comment on thi , as he pleas . thank to all who were help . regard ,


In [ ]:
X_train, X_test, y_train, y_test = stem_train, stem_test, twenty_train.target, twenty_test.target

# Пайплайн для KNN
knn_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())
])

# Параметры для KNN
knn_param_grid = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf__n_neighbors': [3, 5, 7]}

# Пайплайн для RF
rf_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

# Параметры для RF
rf_param_grid = {'vect__ngram_range': [(1, 1), (1, 2)],
                 'tfidf__use_idf': (True, False),
                 'clf__n_estimators': [50, 100, 200]}


# Параметры для LR
lr_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('scaler', StandardScaler(with_mean=False)), 
    ('clf', LogisticRegression(max_iter=1000))
])

# Параметры для  LR
lr_param_grid = {'vect__ngram_range': [(1, 1), (1, 2)],
                 'tfidf__use_idf': (True, False),
                 'clf__C': [0.1, 1, 10]}

# Список параметров
pipelines = [knn_pipeline, rf_pipeline, lr_pipeline]
param_grids = [knn_param_grid, rf_param_grid, lr_param_grid]

# Цикл обучения
for i, pipeline in enumerate(pipelines):
    print(f"Оптимизация параметров для {pipeline.named_steps['clf'].__class__.__name__}")
    
    # Данные со стеммингом
    grid_search_stem = GridSearchCV(pipeline, param_grids[i], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search_stem.fit(X_train, y_train)

    print("Лучшие параметры с использованием стемминга:", grid_search_stem.best_params_)
    y_pred_stem = grid_search_stem.predict(X_test)
    print("Точность с использованием стемминга:", accuracy_score(y_test, y_pred_stem))
    print("Точность с использованием стемминга:", precision_score(y_test, y_pred_stem, average='weighted'))
    print("Полнота с использованием стемминга:", recall_score(y_test, y_pred_stem, average='weighted'))
    print("F1-мера с использованием стемминга:", f1_score(y_test, y_pred_stem, average='weighted'))

    # Данные без стемминга
    grid_search_no_stem = GridSearchCV(pipeline, param_grids[i], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search_no_stem.fit(twenty_train.data, twenty_train.target)

    print("Лучшие параметры без стемминга:", grid_search_no_stem.best_params_)
    y_pred_no_stem = grid_search_no_stem.predict(twenty_test.data)
    print("Точность без стемминга:", accuracy_score(twenty_test.target, y_pred_no_stem))
    print("Точность без стемминга:", precision_score(twenty_test.target, y_pred_no_stem, average='weighted'))
    print("Полнота без стемминга:", recall_score(twenty_test.target, y_pred_no_stem, average='weighted'))
    print("F1-мера без стемминга:", f1_score(twenty_test.target, y_pred_no_stem, average='weighted'))
    print("\n")

Оптимизация параметров для KNeighborsClassifier
Лучшие параметры с использованием стемминга: {'clf__n_neighbors': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Точность с использованием стемминга: 0.5131467345207803
Точность с использованием стемминга: 0.5676424720913685
Полнота с использованием стемминга: 0.5131467345207803
F1-мера с использованием стемминга: 0.5019390442761362
Лучшие параметры без стемминга: {'clf__n_neighbors': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Точность без стемминга: 0.47837150127226463
Точность без стемминга: 0.5142921279053392
Полнота без стемминга: 0.47837150127226463
F1-мера без стемминга: 0.4701362633442179


Оптимизация параметров для RandomForestClassifier
Лучшие параметры с использованием стемминга: {'clf__n_estimators': 200, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Точность с использованием стемминга: 0.8125530110262935
Точность с использованием стемминга: 0.8170167261958017
Полнота с использованием стемминга: